In [ ]:
from funcs.notebook import *

from lightkurve import search_lightcurvefile
from altaipony.lcio import from_mast
from altaipony.flarelc import FlareLightCurve


In [ ]:
ff = pd.read_csv("../results/2020_02_flares_vetted.csv")
ff = ff.drop(["real?","note"], axis=1)
f = ff.dropna().drop_duplicates(subset=["ID", "qcs","istart"])
f

In [ ]:
f.shape, ff.dropna().shape

In [ ]:
gf = f[["ID","qcs","mission",'tstamp']].drop_duplicates(subset=["ID","qcs","mission"])

In [ ]:
gf.shape

In [ ]:
row = gf.iloc[1296]

# plot all flares for a light curve at once
ID, qcs = row.ID, int(row.qcs)
sdf = f[(f.qcs == qcs) & (f.ID == ID) & (f.mission == row.mission)]

lc = from_mast(ID,c=qcs,mission=row.mission,cadence="short",flux_type="PDCSAP_FLUX",
               download_dir="/home/ekaterina/Documents/001_science/lcs")

# make it a list of LCs even if only one LC is returned
if type(lc) == FlareLightCurve:

    lc = [lc]

elif type(lc) == list:

    lc = lc
tts = np.array([(l.time[0], l.time[-1]) for l in lc])
print(row.ID, row.qcs)

In [ ]:
%matplotlib inline
for i, row in sdf.iloc[:].iterrows():
    plt.figure(figsize=(17,5))
    ts, tf = row.tstart, row.tstop
    
    # pick

    if row.mission=="TESS":
        LC = lc[0]
        cad=.5
    else:
        nlc = np.where((ts>tts)[:,0] != (ts>tts)[:,1])[0][0]
        LC = lc[nlc]
        cad = 5.25
    _ = LC[np.where((LC.time>=ts-.2/cad) & (LC.time<=tf+.2/cad))]
    plt.plot(_.time, _.flux, c="k")

    _ = LC[np.where((LC.time>=ts-1e-8) & (LC.time<=tf+1e-8))]
    plt.scatter(_.time, _.flux, c="r")
#     plt.ylim(18000,18500)

In [ ]:
from lightkurve import search_targetpixelfile

In [ ]:
tpf = search_targetpixelfile(ID,sector=qcs,mission="TESS").download_all()

In [ ]:
tpf

In [ ]:
tpf[0].interact(notebook_url='localhost:8889')

In [ ]:
lc[0].plot()
# # # plt.scatter(lc[0].time, lc[0].flux)
# plt.xlim(1323,1323.3)
# plt.ylim(17000,18000);

In [ ]:
2e-3 * 60 * 24

In [ ]:
ff = pd.read_csv("../results/2020_02_flares_vetted.csv")


In [ ]:
_t =1576.37151889061




select = (ff.tstart > _t - 1e-3) & (ff.tstart < _t + 1e-3)

In [ ]:
ff[select].groupby("note").count()

In [ ]:
ff[select].shape

In [ ]:
ff.loc[select, "note"] = "simultaneous FP in multiple Lcs"

In [ ]:
ff.loc[select].groupby("note").count()

In [ ]:
ff.shape

In [ ]:
ff.to_csv("../results/2020_02_flares_vetted.csv", index=False)